In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, callbacks, optimizers, regularizers
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

In [2]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
base_dir = '/content/drive/MyDrive/FishImgDataset'

train_dir = os.path.join(base_dir, 'train')
val_dir   = os.path.join(base_dir, 'val')
test_dir  = os.path.join(base_dir, 'test')

print("Isi folder:")
print(os.listdir(base_dir))

print("Isi folder TRAIN:")
print(os.listdir(os.path.join(base_dir, 'train')))

print("Isi folder VAL:")
print(os.listdir(os.path.join(base_dir, 'val')))

print("Isi folder TEST:")
print(os.listdir(os.path.join(base_dir, 'test')))

Isi folder:
['test', 'train', 'val']
Isi folder TRAIN:
['Goby', 'Black Spotted Barb', 'Bangus', 'Fourfinger Threadfin', 'Climbing Perch', 'Glass Perchlet', 'Big Head Carp', 'Catfish', 'Freshwater Eel', 'Jaguar Gapote', 'Long-Snouted Pipefish', 'Indian Carp', 'Janitor Fish', 'Gold Fish', 'Grass Carp', 'Green Spotted Puffer', 'Knifefish', 'Gourami', 'Indo-Pacific Tarpon', 'Perch', 'Pangasius', 'Mullet', 'Silver Barb', 'Snakehead', 'Scat Fish', 'Silver Perch', 'Mosquito Fish', 'Mudfish', 'Silver Carp', 'Tilapia', 'Tenpounder']
Isi folder VAL:
['Black Spotted Barb', 'Catfish', 'Freshwater Eel', 'Fourfinger Threadfin', 'Climbing Perch', 'Glass Perchlet', 'Bangus', 'Big Head Carp', 'Gold Fish', 'Jaguar Gapote', 'Janitor Fish', 'Indo-Pacific Tarpon', 'Grass Carp', 'Indian Carp', 'Goby', 'Green Spotted Puffer', 'Gourami', 'Knifefish', 'Mullet', 'Silver Perch', 'Silver Barb', 'Mosquito Fish', 'Pangasius', 'Perch', 'Long-Snouted Pipefish', 'Mudfish', 'Silver Carp', 'Scat Fish', 'Tilapia', 'Snake

In [4]:
image_size = (128, 128)
batch_size = 32

datagen = ImageDataGenerator(rescale=1./255)

train_data = datagen.flow_from_directory(train_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical')
val_data = datagen.flow_from_directory(val_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical')
test_data = datagen.flow_from_directory(test_dir, target_size=image_size, batch_size=batch_size, class_mode='categorical', shuffle=False)

Found 8801 images belonging to 31 classes.
Found 2751 images belonging to 31 classes.
Found 1760 images belonging to 31 classes.


In [ ]:
# 2. Data Augmentation & Preprocessing
import os

image_size = (128, 128)
batch_size = 32

base_dir = '/content/drive/MyDrive/FishImgDataset'
train_dir = os.path.join(base_dir, 'train')
val_dir   = os.path.join(base_dir, 'val')
test_dir  = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)

train_data = datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_data = datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_data = datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# 3. CNN Model Architecture dengan L2 Regularization
from tensorflow.keras import regularizers

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3), kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(train_data.num_classes, activation='softmax')
])

# 4. Compile dengan Learning Rate Scheduler
initial_lr = 0.001
lr_schedule = callbacks.LearningRateScheduler(lambda epoch: initial_lr * (0.95 ** epoch))

model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.Adam(learning_rate=initial_lr),
    metrics=['accuracy']
)

# 5. Callbacks
early_stop = callbacks.EarlyStopping(patience=5, restore_best_weights=True)

# 6. Training
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=30,
    callbacks=[early_stop, lr_schedule]
)

# 7. Evaluation
test_data.reset()
pred = model.predict(test_data)
pred_labels = np.argmax(pred, axis=1)
true_labels = test_data.classes

print("\nClassification Report:")
print(classification_report(true_labels, pred_labels))

print("\nConfusion Matrix:")
cm = confusion_matrix(true_labels, pred_labels)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

# 8. ROC & AUC
if train_data.num_classes == 2:
    auc = roc_auc_score(true_labels, pred[:,1])
    fpr, tpr, _ = roc_curve(true_labels, pred[:,1])
    plt.plot(fpr, tpr)
    plt.title(f"ROC Curve (AUC = {auc:.2f})")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.grid()
    plt.show()
else:
    print("ROC Curve hanya berlaku untuk binary classification.")

Found 8801 images belonging to 31 classes.
Found 2751 images belonging to 31 classes.
Found 1760 images belonging to 31 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
276/276 ━━━━━━━━━━━━━━━━━━━━ 0s 17s/step - accuracy: 0.1302 - loss: 3.4706 